## 1. Load & Inspect Data

In [5]:
import pandas as pd

df = pd.read_csv("electricity_crisis_palestine_with_water.csv")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   region            250 non-null    object 
 1   zone              250 non-null    object 
 2   facility_type     250 non-null    object 
 3   hour              250 non-null    int64  
 4   demand_mw         250 non-null    float64
 5   outage_risk       250 non-null    float64
 6   priority_level    250 non-null    int64  
 7   pump_capacity_m3  250 non-null    float64
dtypes: float64(3), int64(2), object(3)
memory usage: 15.8+ KB


## 2. Feature Selection

In [6]:
X = df.drop("demand_mw", axis=1)
y = df["demand_mw"]

## 3. Encode Categorical Features


In [7]:
X_encoded = pd.get_dummies(X)

## 4. Train / Test Split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

## 5. Train AI Model (Random Forest)

In [9]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=120,
    random_state=42
)

model.fit(X_train, y_train)


RandomForestRegressor(n_estimators=120, random_state=42)

## 6. Evaluate Model Performance


In [10]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error (MW):", round(mae, 2))


Mean Absolute Error (MW): 12.47


## 7. Feature Importance

In [11]:
import pandas as pd

importance = pd.Series(
    model.feature_importances_,
    index=X_encoded.columns
).sort_values(ascending=False)

importance.head(10)


priority_level                    0.314943
facility_type_Hospital            0.297923
outage_risk                       0.094691
facility_type_Residential         0.070752
hour                              0.069231
facility_type_Emergency Center    0.041959
pump_capacity_m3                  0.035173
zone_Jenin                        0.011035
zone_Hebron                       0.008901
zone_North Gaza                   0.008254
dtype: float64

## 8. Save Predictions for Quantum Stage

In [12]:
df["predicted_demand_mw"] = model.predict(X_encoded)

df.to_csv("ai_output_for_quantum.csv", index=False)